In [1]:
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.4/534.4 KB 4.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached botocore-1.26.10-py3-none-any.whl (8.8 MB)
  Created wheel for sagemaker: filename=sagemaker-2.96.0-py2.py3-none-any.whl size=747388 sha256=ac84e7302f7e3a2c6c3a9dcd954567258f19cdf072f2573d8f120e49ada2173c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/85/f0/6e/27188ffde539234dc8ab05d24ec0f29162e84352ac6770b6d6
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.92.1
    Uninstalling sagemaker-2.92.1:
      Successfully uninstalled sagemaker-2.92.1
ERROR: pip's dependency resolver does not currently take into account all the packages

In [5]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'
              } # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Matplotlib is building the font cache; this may take a moment.


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [6]:
bucket_name = 'xgboost-imd-youngjin' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [7]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [8]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [9]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [10]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],
                                    role, 
                                    instance_count=2, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)
                                    
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

In [11]:
xgb.fit({'train': s3_input_train})

2022-06-26 14:38:09 Starting - Starting the training job...
2022-06-26 14:38:35 Starting - Preparing the instances for trainingProfilerReport-1656254289: InProgress
.........
2022-06-26 14:40:05 Downloading - Downloading input data......
2022-06-26 14:41:00 Training - Downloading the training image.....Arguments: train
[2022-06-26:14:41:50:INFO] Running distributed xgboost training.
[2022-06-26:14:41:50:INFO] Number of hosts: 2, master IP address: 10.0.178.17, host IP address: 10.0.133.147.
[2022-06-26:14:41:50:INFO] Finished Yarn configuration files setup.
starting datanode, logging to /opt/amazon/hadoop/logs/hadoop--datanode-ip-10-0-133-147.ec2.internal.out
Arguments: train
[2022-06-26:14:41:53:INFO] Running distributed xgboost training.
[2022-06-26:14:41:53:INFO] Number of hosts: 2, master IP address: 10.0.178.17, host IP address: 10.0.178.17.
[2022-06-26:14:41:53:INFO] Finished Yarn configuration files setup.
starting nodemanager, logging to /opt/amazon/hadoop/logs/yarn--nodemanage

In [12]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

----------!

In [13]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [14]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10770)    36% (166)
Purchase        10% (1132)     64% (289) 



In [15]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'PXH29F2XRF3Q75X8',
   'HostId': 'plPUkAMvMEVAsoEexYfENWMqJL3AnTWyjDydBiWcB4bAjVPWYW+HDIEzPDhNmYDhCZjZ4U/FbAs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'plPUkAMvMEVAsoEexYfENWMqJL3AnTWyjDydBiWcB4bAjVPWYW+HDIEzPDhNmYDhCZjZ4U/FbAs=',
    'x-amz-request-id': 'PXH29F2XRF3Q75X8',
    'date': 'Sun, 26 Jun 2022 14:58:00 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-26-14-38-09-088/profiler-output/system/incremental/2022062614/1656254460.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-26-14-38-09-088/profiler-output/system/incremental/2022062614/1656254520.algo-2.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2022-06-26-14-38-09-088/profiler-output/system/incremental/2022062614/1656254460.algo-2.json'},
   {'Key': '

In [22]:
!aws s3 ls s3://xgboost-imd-youngjin/

2022-06-26 15:06:59    5383698 bank_clean.csv
